<a href="https://colab.research.google.com/github/LongBaoNguyen/longbaonguyen.github.io/blob/master/courses/apcsp/image_processing/svd/svd.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>


# Singular Value Decomposition (SVD) and Image Compression


Singular Value Decomposition (SVD) is one of the most important matrix factorizations in linear algebra. Just as integers can be factored into prime numbers, a matrix can be decomposed into a product of simpler matrices that reveal its fundamental structure. In this lab, you will explore what SVD is, how it represents a matrix in terms of its geometric components, and how this decomposition enables us to compress images through low‑rank approximation.

Singular Value Decomposition states that for **any** real matrix  

$$A \in \mathbb{R}^{m \times n}$$  

can be decomposed as:

$$
A = U \, \Sigma \, V^{T}
$$

where:

- $U \in \mathbb{R}^{m \times m}$ is an **orthogonal matrix** (columns are orthonormal),
- $\Sigma \in \mathbb{R}^{m \times n}$ is a (rectangular) **diagonal matrix** whose diagonal entries  
  $$
  \sigma_1 \ge \sigma_2 \ge \cdots \ge \sigma_r > 0
  $$
  are the **singular values**,
- $V \in \mathbb{R}^{n \times n}$ is another **orthogonal matrix**, and $V^T$ is its transpose.

---

## **Low‑Rank Approximation**

A powerful use of SVD is approximating $A$ using only the top $k$ singular values:

$$
A_k = U_k \, \Sigma_k \, V_k^{T}
$$

where:

- $U_k$ contains the first $k$ columns of $U$,
- $\Sigma_k$ contains the top $k$ singular values,
- $V_k^T$ contains the first $k$ rows of $V^T$.

This yields the **best rank-$k$** approximation to $A$.

---

## **Why This Works for Images**

A grayscale image is a matrix of pixel values. By keeping only the largest singular values, we capture the most important structure and discard noise or fine detail.

Thus:

- store fewer numbers,
- reconstruct an image that looks nearly the same,
- achieve **image compression via SVD**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import requests
from io import BytesIO

url = "https://raw.githubusercontent.com/LongBaoNguyen/longbaonguyen.github.io/master/courses/apcsp/datasets/cup.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("L")
img = np.asarray(img).astype(np.float32) / 255.0  # shape: (H, W)
H, W = img.shape

print(f"Loaded image shape: {img.shape}")
plt.figure(figsize=(5,5))
plt.imshow(img, cmap="gray")
plt.title('Original Image')
plt.show()


## Understanding `numpy.linalg.svd` and the Shapes of Its Outputs

Recall that the **Singular Value Decomposition (SVD)** breaks any matrix $A \in \mathbb{R}^{m \times n}$ into three pieces:

$$
A = U \, \Sigma \, V^{T}
$$

where:

- $U \in \mathbb{R}^{m \times m}$, $\Sigma \in \mathbb{R}^{m \times n}$ and $V^T \in \mathbb{R}^{n \times n}$ 

NumPy computes this using:

```python
U, s, Vt = np.linalg.svd(A)
```

However, the sizes of `U`, `s`, and `Vt` uses a **compact** form of SVD by default to save memory. Let $p = \min(m, n)$. NumPy does **not** return the full diagonal matrix $\Sigma.$ Instead, it returns a **1D array** of length $p$. Why? Because the full $\Sigma$ would be mostly zeros:

$$
\Sigma =
\begin{bmatrix}
\sigma_1 & 0 & \cdots \\
0 & \sigma_2 & \cdots \\
\vdots & & \ddots \\
\end{bmatrix}
$$

NumPy returns only the diagonal (the important part!) to avoid storing unnecessary zeros.


In addition, even though the theoretical SVD defines $U$ as an **m × m** orthogonal matrix, NumPy returns only the **first $p$ columns**. Similarly, instead of an **n × n** orthogonal matrix $V$, NumPy returns only the **first $p$ rows**, the matrix of singular vectors that matter for the decomposition.

This decomposition is compact, efficient, and contains all the essential information. These columns are the only ones needed to reconstruct $A$. This is called the **reduced (or thin) SVD**.

## Summary Table

| Piece      | NumPy Output | Shape  | Why This Shape?                           |
| ---------- | ------------ | ------ | ----------------------------------------- |
| $$U$$      | `U`          | (m, p) | Only the first useful singular vectors    |
| $$\Sigma$$ | `s`          | (p,)   | Only diagonals needed; zeros omitted      |
| $$V^T$$    | `Vt`         | (p, n) | Only the essential right singular vectors |



## Reconstructing the Matrix

Since NumPy doesn’t give the full $\Sigma$, you rebuild it like this:

```python
Sigma = np.diag(s)   # creates a p x p diagonal matrix
A_reconstructed = U @ Sigma @ Vt
```

This produces a matrix equal (up to numerical precision) to the original $A$.



#### Suppose the image's shape is 600 x 800. To store the original image, how many entries are stored in memory? If k = 50, how many entries are stored in the compressed image?

#### Answer here:

#### Fill out code below by choosing only the top k singular values in the decomposition. Which k is acceptable in terms of quality? Try k = 5, 10, 20, 40, 50, 100.

In [ ]:
k = 20
# call svd
U, s, Vt = 

# first k columns of U 

# first k singular values of s 

# first k rows of Vt 


# reconstruct A by using approximated Ak


# clipped values, use np.clip(Ak, 0, 1)
Ak = np.clip(Ak, 0, 1)


plt.figure(figsize=(5,5))
plt.imshow(Ak, cmap="gray")
plt.title(f'Compressed Image k={k}')
plt.show()



## Compression math: how many numbers do we store?

- Original grayscale image stores $m\times n$ numbers.
- Rank-$k$ SVD approximation stores:
  - $U_k$: $m\times k$
  - $\Sigma_k$: $k$ (just the diagonal)
  - $V_k^T$: $k\times n$

Total stored numbers: $k(m+n+1)$

Compression ratio (approx):
$$\text{ratio} = \dfrac{mn}{k*(m+n+1)}$$


In [ ]:
def compression_ratio(m, n, k):
    # fill in code to return the ratio above.(one line, fill in formula after "return")
    return 
    
    

for k in [5,10,20,50,100]:
    print(f"k={k:>3}  ratio≈{compression_ratio(H, W, k):.2f}x fewer numbers")


## Optional: Interactive slider (if `ipywidgets` is available)

If you're in Jupyter with widgets enabled (or Colab with widgets support), you can use a slider to change \(k\) live.
If it errors, you can skip this section.


In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display

    k_slider = widgets.IntSlider(value=1, min=1, max=100, step=1, description='k')

    out = widgets.Output()

    def update(change=None):
        with out:
            out.clear_output(wait=True)
            k = k_slider.value
            Ak = U[:,:k] @ np.diag(s[:k]) @ Vt[:k,:]
            Ak = np.clip(Ak, 0, 1)
            plt.figure(figsize=(5,5))
            plt.imshow(Ak, cmap="gray")
            plt.axis('off')
            plt.title(f'Rank-k RGB Approximation (k={k})')
            plt.show()
            print(f"Approx compression ratio ≈ {compression_ratio(H, W, k):.2f}x")

    k_slider.observe(update, names='value')
    display(k_slider, out)
    update()

except Exception as e:
    print("ipywidgets not available or not enabled. Skipping interactive section.")
    print("Error:", e)



## SVD compression for *color* images (RGB)

A color image has 3 channels (Red, Green, Blue). One simple approach is:

1. Compute SVD separately on each channel matrix
2. Keep the top \(k\) components for each channel
3. Reconstruct and stack back into an RGB image


In [ ]:
url = "https://raw.githubusercontent.com/LongBaoNguyen/longbaonguyen.github.io/master/courses/apcsp/datasets/flower.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")
img_rgb = np.asarray(img).astype(np.float32) / 255.0  # shape: (H, W, 3)
H, W, D = img_rgb.shape

print(f"Loaded image shape: {img_rgb.shape}")
plt.figure(figsize=(5,5))
plt.imshow(img_rgb)
plt.title('Original Image')
plt.show()



### Reconstruct RGB using rank-\(k\) per channel

We'll compare reconstructions for different \(k\). For each \(k\), we compute:

$$
R_k = U^R_k\,\Sigma^R_k\,(V^R_k)^T,\quad
G_k = U^G_k\,\Sigma^G_k\,(V^G_k)^T,\quad
B_k = U^B_k\,\Sigma^B_k\,(V^B_k)^T
$$

and stack them into an RGB image.


In [ ]:
# extract the Red, Green, Blue components from image, use slicing
R = 
G = 
B = 

# compute SVD for each channel, call np.linalg.svd(channel, full_matrices=False)
Ur, sr, Vtr = 
Ug, sg, Vtg =
Ub, sb, Vtb = 

print("SVD computed for each channel.")


def svd_approx(U, s, Vt, k):
    # U: (m,kmax), s: (kmax,), Vt: (kmax,n)
    # return (m,n)
    Ak = U[:,:k] @ np.diag(s[:k]) @ Vt[:k,:]
    return Ak


def rgb_svd_approx(k):
    # call svd_approx for each channel
    Rk = 
    Gk = 
    Bk = 
    out = np.stack([Rk, Gk, Bk], axis=-1)
    return np.clip(out, 0, 1)



# No need to write code below this line

ks_rgb = [5, 10, 20, 50, 100]
ks_rgb = [k for k in ks_rgb if k <= min(H, W)]

fig, axes = plt.subplots(1, len(ks_rgb) + 1, figsize=(3*(len(ks_rgb)+1), 3))
axes[0].imshow(img_rgb)
axes[0].set_title('Original')
axes[0].axis('off')

for i, k in enumerate(ks_rgb, start=1):
    img_k = rgb_svd_approx(k)
    axes[i].imshow(img_k)
    axes[i].set_title(f'k={k}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()



## Storage comparison for RGB

Original RGB stores $3mn$ numbers.
For rank-$k$ per channel, we store for each channel $k(m+n+1)$, so total:

$$
3k(m+n+1)
$$

Compression ratio (approx):
$$
	\text{ratio}_{RGB} = \dfrac{3mn}{3k(m+n+1)} = \dfrac{mn}{k(m+n+1)}
$$

So the same formula applies (when using the same \(k\) for each channel).


In [ ]:
for k in ks_rgb:
    print(f"k={k:>3}  ratio≈{compression_ratio(H, W, k):.2f}x fewer numbers (approx)")



## Optional: Interactive slider (if `ipywidgets` is available)

If you're in Jupyter with widgets enabled (or Colab with widgets support), you can use a slider to change \(k\) live.
If it errors, you can skip this section.


In [ ]:
# just run this cell and play around with slider, no extra code needed

try:
    import ipywidgets as widgets
    from IPython.display import display

    k_slider = widgets.IntSlider(value=1, min=1, max=100, step=1, description='k')

    out = widgets.Output()

    def update(change=None):
        with out:
            out.clear_output(wait=True)
            k = k_slider.value
            img_k = rgb_svd_approx(k)
            plt.figure(figsize=(5,5))
            plt.imshow(img_k)
            plt.axis('off')
            plt.title(f'Rank-k RGB Approximation (k={k})')
            plt.show()
            print(f"Approx compression ratio ≈ {compression_ratio(H, W, k):.2f}x")

    k_slider.observe(update, names='value')
    display(k_slider, out)
    update()

except Exception as e:
    print("ipywidgets not available or not enabled. Skipping interactive section.")
    print("Error:", e)
